## start with Creating tf vectorizer
   ### Require creating pre processing pipeline that contane (remove stopwords)
    

In [1]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def lowercasing(text):
    return text.lower()

def remove_punctuation(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

def tokenize(text):
    return word_tokenize(text)

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word.lower() not in stop_words]

def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in tokens]
def join_token(token):
    return ' '.join(token)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Haboubi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Haboubi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Haboubi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### *Create preprocessing pipeline 

In [2]:
def preprocessing_pipeline(txt):
    lower=lowercasing(txt)
    remove_punc=remove_punctuation(lower)
    token=tokenize(remove_punc)
    remove_stop=remove_stopwords(token)
    lemma=lemmatize(remove_stop)
    return join_token(lemma)
sample_text = "The quick brown fox jumps over the lazy dog."

preprocessing_pipeline(sample_text)   

'quick brown fox jump lazy dog'

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Sample data
corpus = 'This is the first document . This document is the second document And this is the third one. Is this the first document?'

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 2))

new_text = "This document discusses various aspects of natural language processing."

# Fit and transform the data
tfidf_matrix = tfidf_vectorizer.fit_transform([preprocessing_pipeline(corpus)])
new_text_tfidf = tfidf_vectorizer.transform([preprocessing_pipeline(new_text)])

# Print the TF-IDF matrix
print(tfidf_matrix.toarray())

def tfidf_matrix(text):
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 2))
    tfidf_matrix = tfidf_vectorizer.fit_transform([preprocessing_pipeline(text)])
    return tfidf_vectorizer
    

    

[[0.31622777 0.31622777 0.31622777 0.63245553 0.31622777 0.31622777
  0.31622777]]


In [4]:
new_text_tfidf.toarray()

array([[0., 0., 0., 0., 0., 0., 0.]])

In [5]:
import numpy as np
def extract_keywords(tfidf_vectorizer, text, top_n=5):
    tfidf_scores = tfidf_vectorizer.transform([preprocessing_pipeline(text)])
    feature_names=tfidf_vectorizer.get_feature_names_out()
    print(tfidf_scores)

    top_indices = tfidf_scores.argsort()[0, ::-1][:top_n]
    
    keywords = [feature_names[i] for i in top_indices]
    return keywords

In [6]:
#Functions
# Extract text with PyPDF2
from PyPDF2 import PdfReader,PdfWriter
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
import pdfplumber
import fitz
pdf_name='PMBOK6-2017.pdf'

def extract_text_from_pdf_PyPDF2(pdf_name,debut=0,end=0):
    text=''
    pdf_file=open(pdf_name,'rb')
    pdf=PyPDF2.PdfReader(pdf_file)
    if end==0:
        end=len(pdf.pages)-1
    for pg_num in range(debut,end+1):
        page=pdf.pages[pg_num]
        text += page.extract_text ()
    return  text   
# Extract selected pages as pdf
def get_selected_pages_as_pdf(pdf_path,start,end):
    with open(pdf_path,'rb') as f:
        all_pdf=PdfReader(f)
        write_pdf=PdfWriter()
        new_pdf_name=f"{pdf_path}_from_page_{start}_{end}.pdf"
        if end!=len(all_pdf.pages):
            end+=1
        
        for page_num in range(start-1,end-1):
            pages=all_pdf.pages[page_num]
            write_pdf.add_page(pages)
        with open (new_pdf_name,'wb') as out:
            write_pdf.write(out)    

# Extract text with  pdfminer
def extract_text_from_pdf_pdfminer(pdf_path):
    laparams = LAParams()
    text = extract_text(pdf_path,laparams=laparams)
    return text

def organaize_title_with_sub_title(liste_titre,liste_titre_sub):
    dic_titre_position={}
    dic_titre={}
    liste_titre_position=[]
    liste_only_titre=[]
    for index1 in range(len(liste_titre)) :
        for index in range(len(liste_titre_sub)):
            space=liste_titre[index1][0].find(' ')
            if liste_titre_sub[index][0].find(liste_titre[index1][0][:space])>=0:
                liste_titre_position.append(liste_titre_sub[index])
                
                liste_only_titre.append(liste_titre_sub[index][0])
        
        dic_titre_position[liste_titre[index1]]=liste_titre_position
        
        dic_titre[liste_titre[index1][0]]=liste_only_titre
        
        liste_titre_position=[]
        
        liste_only_titre=[]
        
    return dic_titre,dic_titre_position
def sort_liste_based_on_element(liste,element):
        sorted_list=sorted(liste,key=lambda x:x[element])
        return sorted_list
    

def get_liste_of_slice(dic_position_liste,last_end=-1): #require a dictionary that contains the position of each title
    dic_contenu={}
    liste=[]
    dic_position_liste=list(dic_position_liste)
    dic_position_liste=sort_liste_based_on_element(dic_position_liste,1)#sort liste_element based on second element example ('11.1',102)>('11.2',106)
    for i in range(len(dic_position_liste)):
        start_position=int(dic_position_liste[i][1])
        if i!=len(dic_position_liste)-1:
            end_position=int(dic_position_liste[i+1][1])
        else:
            end_position=last_end
        liste.append([start_position,end_position])
    
         
        
    return liste





def get_contenu(dic_position,text): # require a dictionary that contains the position of each title
    dic_contenu={}
    liste_slice=get_liste_of_slice(dic_position)
    print(liste_slice)
    for i in range(len(liste_slice)):
        start_position=liste_slice[i][0]
        end_position=liste_slice[i][1]
        dic_contenu[list(dic_position.keys())[i][0]]=text[start_position:end_position]
    return dic_contenu

def get_detailed_contenu(dic_position_title,text): # require same as get_contenu , but it dig deeper and get content of sub_titles
    
    liste_of_keys=list(dic_position_title.keys())
    dic_contenu={}
    
    for index in range(len(liste_of_keys)):
        
        liste_value_per_key=dic_position_title[liste_of_keys[index]]
        if index <=len(liste_of_keys)-2:
            last_position=liste_of_keys[index+1][1]
        else:
            last_position=-1
        liste_slice_of_value=get_liste_of_slice(liste_value_per_key,last_position)
            
        for i in range(len(liste_slice_of_value)):
            start_position=liste_slice_of_value[i][0]
            end_position=liste_slice_of_value[i][1]
            dic_contenu[liste_value_per_key[i][0]]=text[start_position:end_position]
    return dic_contenu    
    
    
    
# Extract tables using pdfplumber

def extract_tables_pdfplumber(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_number in range(len(pdf.pages)):
            page = pdf.pages[page_number]
            page_tables = page.extract_tables()
            if page_tables:
                for table in page_tables:
                    tables.append(table)
    
    return tables
    return tables    
    
    
def extract_text_from_pdf_usingfitz(pdf_path):
    doc = fitz.open(pdf_path)
    text = ''

    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()

    doc.close()
    return text

In [7]:
# apply tfidf vectorizer on extracted data

intro_name='PMBOK6-2017.pdf_from_page_431_436.pdf'
chapiter_name='PMBOK6-2017.pdf_from_page_437_494.pdf'
# Extract text from new selected pages {chapiter_name}
extracted_text = extract_text_from_pdf_usingfitz(chapiter_name)


In [8]:
tfid=tfidf_matrix(extracted_text)

In [9]:
text_f='ENTERPRISE ENVIRONMENTAL FACTORS The enterprise environmental factors that can inﬂuence the Plan Risk Management process include but are not limited to overall risk thresholds set by the organization or key stakeholders .'
f=extract_keywords(tfid,text_f)

  (0, 5479)	0.20851441405707477
  (0, 5052)	0.20851441405707477
  (0, 4874)	0.20851441405707477
  (0, 4790)	0.20851441405707477
  (0, 3945)	0.20851441405707477
  (0, 3750)	0.20851441405707477
  (0, 3556)	0.20851441405707477
  (0, 3477)	0.20851441405707477
  (0, 2991)	0.20851441405707477
  (0, 2872)	0.20851441405707477
  (0, 2779)	0.20851441405707477
  (0, 2734)	0.20851441405707477
  (0, 2513)	0.20851441405707477
  (0, 2121)	0.20851441405707477
  (0, 2119)	0.20851441405707477
  (0, 1918)	0.41702882811414954
  (0, 1909)	0.41702882811414954


AttributeError: argsort not found

In [ ]:
import spacy

def extract_entities(text):
    # Load the spaCy English language model
    nlp = spacy.load("en_core_web_sm")
    
    # Process the text using spaCy
    doc = nlp(text)
    
    # Extract noun phrases (entities)
    entities = [chunk.text for chunk in doc.noun_chunks]
    
    return entities

# Example usage
text = "The enterprise environmental factors that can influence the Plan Risk Management process include but are not limited to overall risk thresholds set by the organization or key stakeholders."
entities = extract_entities(text)
print("Entities:", entities)

In [10]:
def extract_key_words_relationhsip(txt,relationship='influence',verb=True):
    ADP=False
    liste_word=[]
    word=''
    txt=normalize_text(txt)
    txt=txt.replace('\n','')
    doc = nlp(txt)
    save_key_sentence=False
    still_word=False
    
    
    print(txt)
    for token in doc :
        
        if( token.pos_ == 'VERB' and token.text=='influence') or (token.text=='components'):
            ADP=True
        if ADP==True and token.pos_ == 'ADP':
            
            save_key_sentence=True
        if save_key_sentence==True or still_word==True:
            if token.pos_!='PRON' and token.pos_!='PUNCT ' :
                
                word=word+' '+token.text
                print(word)
            
            else:
                save_key_sentence==False
                liste_word.append(word)
                word=''
    if word !='':
        liste_word.append(word)
    return liste_word  

def verify_mutiple_input(text):
    if ':' in normalize_text(text):
        return True
    else:
        return False 

def Extract_inputs_sub_process(text):
    input_string='considered as inputs'
    if input_string in text:
        print('Extract input')

        if verify_mutiple_input(text):
            input_liste=extract_key_word_content(text)[0]
            print(input_liste)
            return input_liste
        
    return ''
def Extract_components_sub_process(text):
    components_string='components'

    if components_string in text :
        print('Extract components')

        if verify_mutiple_input(text):
            input_liste=extract_key_word_content(text)[0]
            
            if input_liste ==[]:
                input_liste=extract_key_words_relationhsip(text,'components')
            print(input_liste)
    
            return input_liste
        
    return None

def Extract_technologies_sub_process(text):
    components_string='techniques'

    if components_string in text and 'include' in text:
        print('Extract techniques')

        if verify_mutiple_input(text):
            input_liste=extract_key_word_content(text)[0]
            print(input_liste)
            
            return input_liste
    return None    
def Extract_Figure(text):
    regex=r'Figure \d+-\d+'
    pattern=re.compile(regex)
    liste=[]
    matchs=pattern.finditer(text)
    for match in matchs:
        liste.append(match.group())
    return set(liste)

def Extract_Table(text):
    regex=r'Table \d+-\d+'
    pattern=re.compile(regex)
    liste=[]
    matchs=pattern.finditer(text)
    for match in matchs:
        liste.append(match.group())
    return set(liste)        
    return None

def Extract_influence_relationship(text):
    influence_string='that can influence'

    if influence_string in normalize_text(text) :
        print('Extract influence')

        if verify_mutiple_input(text):
            input_liste=extract_key_word_content_v2(text)
            if input_liste ==[]:
                input_liste=extract_key_words_relationhsip(text)
                print(input_liste)

            print(input_liste)
            
            return input_liste
    return None

# Extract key_words:
def extract_key_word_content(txt):
    #regex=r'(uu).*(\.\n)' regex using text extracted from pypdf
    regex=r"u\nu.*?(\.)"
    pattern=re.compile(regex)
    matchs=pattern.finditer(txt)
    key_word=[]
    detailed={}
    for match in matchs:
        string= match.group()
        if string.find('.')>0 and string.find('.')<len(string):
            
            detail_debut=match.span()[1]
            detail_end=txt[detail_debut:].find('.\n')
            key=string[:match.group().find('.')].replace('u\nu','')
            key_word.append(key)
            detailed[key]=txt[detail_debut:detail_debut+detail_end]
            
        else:
            key_word.append(string)
    return key_word,detailed

def extract_key_word_content_v2(txt):
    #regex=r'(uu).*(\.\n)' regex using text extracted from pypdf
    regex=r"u\nu.*?(\.|,)"
    pattern=re.compile(regex)
    matchs=pattern.finditer(txt)
    key_word=[]
    detailed={}
    for match in matchs:
        string= match.group()
        
            
            
        key_word.append(string.replace('u\nu',''))
            
        
    return key_word        

In [11]:
import spacy
def filter_single_word_entities(entity_dict):
    filtered_entities = {entity: verb for entity, verb in entity_dict.items() if len(entity.split()) > 1}
    return filtered_entities


def remove_stopwords_from_entities(entity_dict):
    new_dic={}
    for entites,relation in entity_dict.items():
        print(entites)
        new_key=join_token(remove_stopwords(tokenize(entites)))
        print(new_key)
        new_dic[new_key]=relation
    return new_dic   
def clean_entites_verb(entity_verbs):
    entity_verbs_news={}
    for key,val in  entity_verbs.items():
        if isinstance(val, list)==True:
            entity_verbs_news[key]=val[0]
        else:
            entity_verbs_news[key]=val
    return entity_verbs_news 
def extract_entities_with_relationship(text):
    nlp = spacy.load("en_core_web_sm")
    
    doc = nlp(text)
    
    entity_verbs = {}
    
    for chunk in doc.noun_chunks:
        entity = chunk.text
         
        main_verb = None
        for token in chunk.root.ancestors:
            if token.pos_ == "VERB":
                main_verb = token.text
                break
        for token in chunk.root.children:
            if token.pos_ == "VERB":
                main_verb = token.text
                break        
        
        if main_verb:
            entity_verbs[entity] = main_verb
    return clean_entites_verb(entity_verbs)



In [12]:
def convert_dic(entites,extracted,relatonship):
    if extracted==None:
        return entites
    for e in extracted:
        entites[e]=relatonship
    return entites
import unicodedata

def clean_text(input_text):
    cleaned_text = input_text.replace('u\nu', ' ').replace('/n','').replace('  ','').strip()
    cleaned_text = ' '.join(cleaned_text.split())
    cleaned_text=clean_numeration(cleaned_text,3)
    return cleaned_text




def normalize_text(text):
    return unicodedata.normalize('NFKC', text)


def nlp_pipeline_entites_extraction(text):
    text=normalize_text(text)
    clean=clean_text(text)
    entites_relation={}
    # first layer extract :
    entites_relation=extract_entities_with_relationship(clean)
    print(entites_relation)
    # second layer extract influence_rela
    extracted_influence=Extract_influence_relationship(text)
    entites_relation=convert_dic(entites_relation,extracted_influence,'influence')
    
    # third layer extract input_relationship
    extracted_input=Extract_inputs_sub_process(text)
    entites_relation=convert_dic(entites_relation,extracted_input,'input')
    # fourth layer extract conponment_relationship
    
    extracted_components=Extract_components_sub_process(text)
    entites_relation=convert_dic(entites_relation,extracted_components,'components')
    # Fifth layer extract thecnique_relationship
    
    extracted_tech=Extract_technologies_sub_process(text)
    entites_relation=convert_dic(entites_relation,extracted_tech,'techniques')
    return entites_relation  

In [13]:
# extract txt from corpse
#Functions
# Extract text with PyPDF2
import PyPDF2
from PyPDF2 import PdfReader,PdfWriter
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
import pdfplumber
import fitz
pdf_name='PMBOK6-2017.pdf'

def extract_text_from_pdf_PyPDF2(pdf_name,debut=0,end=0):
    text=''
    pdf_file=open(pdf_name,'rb')
    pdf=PyPDF2.PdfReader(pdf_file)
    if end==0:
        end=len(pdf.pages)-1
    for pg_num in range(debut,end+1):
        page=pdf.pages[pg_num]
        text += page.extract_text ()
    return  text   
# Extract selected pages as pdf
def get_selected_pages_as_pdf(pdf_path,start,end):
    with open(pdf_path,'rb') as f:
        all_pdf=PdfReader(f)
        write_pdf=PdfWriter()
        new_pdf_name=f"{pdf_path}_from_page_{start}_{end}.pdf"
        if end!=len(all_pdf.pages):
            end+=1
        
        for page_num in range(start-1,end-1):
            pages=all_pdf.pages[page_num]
            write_pdf.add_page(pages)
        with open (new_pdf_name,'wb') as out:
            write_pdf.write(out)    

# Extract text with  pdfminer
def extract_text_from_pdf_pdfminer(pdf_path):
    laparams = LAParams()
    text = extract_text(pdf_path,laparams=laparams)
    return text

def organaize_title_with_sub_title(liste_titre,liste_titre_sub):
    dic_titre_position={}
    dic_titre={}
    liste_titre_position=[]
    liste_only_titre=[]
    for index1 in range(len(liste_titre)) :
        for index in range(len(liste_titre_sub)):
            space=liste_titre[index1][0].find(' ')
            if liste_titre_sub[index][0].find(liste_titre[index1][0][:space])>=0:
                liste_titre_position.append(liste_titre_sub[index])
                
                liste_only_titre.append(liste_titre_sub[index][0])
        
        dic_titre_position[liste_titre[index1]]=liste_titre_position
        
        dic_titre[liste_titre[index1][0]]=liste_only_titre
        
        liste_titre_position=[]
        
        liste_only_titre=[]
        
    return dic_titre,dic_titre_position
def sort_liste_based_on_element(liste,element):
        sorted_list=sorted(liste,key=lambda x:x[element])
        return sorted_list
    

def get_liste_of_slice(dic_position_liste,last_end=-1): #require a dictionary that contains the position of each title
    dic_contenu={}
    liste=[]
    dic_position_liste=list(dic_position_liste)
    dic_position_liste=sort_liste_based_on_element(dic_position_liste,1)#sort liste_element based on second element example ('11.1',102)>('11.2',106)
    for i in range(len(dic_position_liste)):
        start_position=int(dic_position_liste[i][1])
        if i!=len(dic_position_liste)-1:
            end_position=int(dic_position_liste[i+1][1])
        else:
            end_position=last_end
        liste.append([start_position,end_position])
    
         
        
    return liste





def get_contenu(dic_position,text): # require a dictionary that contains the position of each title
    dic_contenu={}
    liste_slice=get_liste_of_slice(dic_position)
    print(liste_slice)
    for i in range(len(liste_slice)):
        start_position=liste_slice[i][0]
        end_position=liste_slice[i][1]
        dic_contenu[list(dic_position.keys())[i][0]]=text[start_position:end_position]
    return dic_contenu

def get_detailed_contenu(dic_position_title,text): # require same as get_contenu , but it dig deeper and get content of sub_titles
    
    liste_of_keys=list(dic_position_title.keys())
    dic_contenu={}
    
    for index in range(len(liste_of_keys)):
        
        liste_value_per_key=dic_position_title[liste_of_keys[index]]
        if index <=len(liste_of_keys)-2:
            last_position=liste_of_keys[index+1][1]
        else:
            last_position=-1
        liste_slice_of_value=get_liste_of_slice(liste_value_per_key,last_position)
            
        for i in range(len(liste_slice_of_value)):
            start_position=liste_slice_of_value[i][0]
            end_position=liste_slice_of_value[i][1]
            dic_contenu[liste_value_per_key[i][0]]=text[start_position:end_position]
    return dic_contenu    
    
    
    
# Extract tables using pdfplumber

def extract_tables_pdfplumber(pdf_path):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_number in range(len(pdf.pages)):
            page = pdf.pages[page_number]
            page_tables = page.extract_tables()
            if page_tables:
                for table in page_tables:
                    tables.append(table)
    
    return tables
    return tables    
    
    
def extract_text_from_pdf_usingfitz(pdf_path):
    doc = fitz.open(pdf_path)
    text = ''

    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()

    doc.close()
    return text


In [14]:
import re 

regex_title=r"11(\.\d){1}\ .+"
regex_sub_title=r"11(\.\d){2}\ .+"
regex_sub2_title=r"11(\.\d){3}\ .+" # this for sub_sub_title example : 11.1.1.1
all_liste=[]
liste_titre=[]
liste_sub_title=[]
liste_sub2_title=[]
for regex in [regex_title,regex_sub_title,regex_sub2_title]:
    liste=[]
    pattern = re.compile(regex)

    matches=pattern.finditer(extracted_text)
    for match in matches :
        liste.append((match.group(),match.start()))
    all_liste.append([liste])
liste_titre=all_liste[0][0]
liste_sub_title=all_liste[1][0] 
liste_sub2_title =all_liste[2][0]  
all_liste=[]
print(liste_titre[0][0])

11.1 PLAN RISK MANAGEMENT


In [15]:
dic_title,dic_title_position=organaize_title_with_sub_title(liste_titre,liste_sub_title)

In [16]:
dic_sub_title,dic_sub_title_position=organaize_title_with_sub_title(liste_sub_title,liste_sub2_title)

In [17]:
dict_contenu=get_contenu(dic_title_position,extracted_text)

[[48, 12474], [12474, 34254], [34254, 51201], [51201, 72011], [72011, 100705], [100705, 108098], [108098, -1]]


In [18]:
#get_liste_of_slice(dic_title_position.keys())
liste=sort_liste_based_on_element(list(dic_title_position.keys()),0)
liste

[('11.1 PLAN RISK MANAGEMENT', 48),
 ('11.2 IDENTIFY RISKS', 12474),
 ('11.3 PERFORM QUALITATIVE RISK ANALYSIS', 34254),
 ('11.4 PERFORM QUANTITATIVE RISK ANALYSIS', 51201),
 ('11.5 PLAN RISK RESPONSES', 72011),
 ('11.6 IMPLEMENT RISK RESPONSES', 100705),
 ('11.7 MONITOR RISKS', 108098)]

In [19]:
detailed_contenu=get_detailed_contenu(dic_sub_title_position,extracted_text)
detailed_contenu['11.1.1.2 PROJECT MANAGEMENT PLAN']

'11.1.1.2 PROJECT MANAGEMENT PLAN\nDescribed in Section 4.2.3.1. In planning Project Risk Management, all approved subsidiary management plans \nshould be taken into consideration in order to make the risk management plan consistent with them. The methodology \noutlined in other project management plan components might inﬂuence the Plan Risk Management process.\n'

In [20]:
import re

def Extract_refenrence(text):
    regex=r'Described in Section.+\.\s'
    pattern=re.compile(regex)
    liste_ref=[]
    matchs=pattern.finditer(text)
    for match in matchs:
        liste_ref.append(match.group())
    return liste_ref


import spacy

# Load English tokenizer, POS tagger, parser, and NER
nlp = spacy.load("en_core_web_sm")

def extract_actionable_items(text):
    doc = nlp(text)
    
    actionable_items = []
    
    for sent in doc.sents:
        if any(token.pos_ == "VERB" for token in sent):
            if sent[0].tag_ == "VB" or sent[0].tag_ == "VBG":
                
                actionable_items.append(sent.text)
    
    return actionable_items

# Example text
example_text = 'make the risk management plan consistent with them. The methodology outlined in other project management plan components might inﬂuence the Plan Risk Management process.'

# Extract actionable items
actionable_items = extract_actionable_items(example_text)

def normalize_text(text):
    return unicodedata.normalize('NFKC', text)


def string_in_liste(string,liste):
    statue=False
    i=0
    while (statue==False and i<len(liste)):
        
        if  liste[i] in normalize_text(string):
            statue=True
        else:
            i+=1
    return statue
        
        
         


def extract_definition(dictionary,text):
    doc = nlp(text)
    defintion=[]
    for sent in doc.sents:
        
        if any(normalize_text(token.lemma_) in dictionary for token in sent): 
            defintion.append(sent)
            break
            
    return defintion      

def Extract_example(text):
    example=[]
    regex=r'\b(for example|such as|include|as an example).*\.'
    pattern=re.compile(regex)
    
    matchs=pattern.finditer(text)
    for match in matchs :
        debut=match.span()[0]
        end=match.span()[1]
        sample=text[:debut]
        last_period_mark=sample[::-1].find('.')
        if last_period_mark!=-1:
            real_debut=debut-last_period_mark
        else:
            real_debut=debut
        example.append(text[real_debut:end])
    return example 

def Extract_Usercases(text):
    Usercases=[]
  
    regex=r'\b(use case|scenario|application|situation).*\.'
    pattern=re.compile(regex)
    
    matchs=pattern.finditer(text)
    for match in matchs :
        debut=match.span()[0]
        end=match.span()[1]
        sample=text[:debut]
        last_period_mark=sample[::-1].find('.')
        if last_period_mark!=-1:
            real_debut=debut-last_period_mark
        else:
            real_debut=debut
        Usercases.append(text[real_debut:end])
    return Usercases   
def string_in_liste(string,liste):
    statue=False
    i=0
    while (statue==False and i<len(liste)):
        
        if  liste[i] in normalize_text(string):
            statue=True
        else:
            i+=1
    return statue
def parent_process_relationship(dic_title,txt):
    for key,value in dic_title.items():
        if txt in value:
            parent,relationship=key.split(':')
            return clean_numeration(parent,2),relationship 
def clean_numeration(text,level=1):
    regex_title=r"11(\.\d){1}\s"
    regex_sub_title=r"11(\.\d){2}\s"
    regex_sub2_title=r"11(\.\d){3}\s" # this for sub_sub_title example : 11.1.1.1
    if level==1:
        regex=regex_title
        pattern = re.compile(regex)

        matches=pattern.finditer(text)
        for m in matches:
            return text.replace(m.group(),'')
    if level==2:
        regex=regex_sub_title
        
        pattern = re.compile(regex)

        matches=pattern.finditer(text)
        for m in matches:
            return text.replace(m.group(),'')
    if level==3:
        regex=regex_sub2_title
   
        pattern = re.compile(regex)

        matches=pattern.finditer(text)
        for m in matches:
            return text.replace(m.group(),'')         
        
         


def extract_definition(dictionary,text):
    doc = nlp(text)
    defintion=[]
    for sent in doc.sents:
        
        if any(normalize_text(token.lemma_) in dictionary for token in sent): 
            defintion.append(sent)
            break
            
    return defintion 

In [25]:
import pandas as pd
dataframe_contenu=pd.DataFrame(detailed_contenu.items(),columns=['sub_title','contenu'])

In [26]:
dataframe_contenu.head()

,sub_title,contenu
0,11.1.1.1 PROJECT CHARTER,11.1.1.1 PROJECT CHARTER\nDescribed in Section...
1,11.1.1.2 PROJECT MANAGEMENT PLAN,11.1.1.2 PROJECT MANAGEMENT PLAN\nDescribed in...
2,11.1.1.3 PROJECT DOCUMENTS,11.1.1.3 PROJECT DOCUMENTS\nProject documents ...
3,11.1.1.4 ENTERPRISE ENVIRONMENTAL FACTORS,11.1.1.4 ENTERPRISE ENVIRONMENTAL FACTORS\nThe...
4,11.1.1.5 ORGANIZATIONAL PROCESS ASSETS,11.1.1.5 ORGANIZATIONAL PROCESS ASSETS\nThe or...


In [27]:
dataframe_contenu.to_csv('sub_process.csv',header=True,index=False)

In [ ]:
# Create dictionary of lemmatized define synonym
dic="define definition setting identify defining set establishing classifying specifying means meaning"
doc = nlp(dic)
liste=[]
for token in doc:
    liste.append(str(token.lemma_))
definition_dic=list(set(liste))

In [ ]:
dataframe_contenu['examples']=dataframe_contenu['contenu'].apply(lambda txt:Extract_example(txt))
dataframe_contenu['definitions']=dataframe_contenu['contenu'].apply(lambda txt:extract_definition(definition_dic,txt))
dataframe_contenu['actionable_items']=dataframe_contenu['contenu'].apply(lambda txt:extract_actionable_items(txt))
dataframe_contenu['Extract_refenrence']=dataframe_contenu['contenu'].apply(lambda txt:Extract_refenrence(txt))
dataframe_contenu.head()

In [ ]:
# extract entites_with_relatonship
dataframe_contenu['entites_relationship_predict']=dataframe_contenu['contenu'].apply(lambda txt:nlp_pipeline_entites_extraction(txt))


In [ ]:
#Evaluation metrics
import pickle
# bring annotated data
file_path = 'evalution_metric.pkl'

with open(file_path, 'rb') as f:
    loaded_list = pickle.load(f)

print("Loaded list:", loaded_list)

In [ ]:
dataframe_contenu['entites_relationship']=loaded_list

In [ ]:
def calculate_scores(true_entities, predicted_entities):
    
    # Calculate true positives, false positives, and false negatives
    true_positives = len(set(true_entities).intersection(predicted_entities))
    false_positives = len(predicted_entities) - true_positives
    false_negatives = len(true_entities) - true_positives
    
    # Calculate precision
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0.0
    
    # Calculate recall
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0.0
    
    # Calculate F1 score
    f1_score = 2 * ((precision * recall) / (precision + recall)) if (precision + recall) != 0 else 0.0
    
    return precision, recall, f1_score

In [ ]:
dataframe_contenu[['precision', 'recall', 'f1_score']] = dataframe_contenu.apply(lambda row: calculate_scores(list(row['entites_relationship'].keys()), list(row['entites_relationship_predict'].keys())), axis=1, result_type='expand')


In [ ]:
dataframe_contenu['entites_relationship'][10].values()

In [ ]:
dataframe_contenu[['precision', 'recall', 'f1_score']]

In [ ]:
#Evaluate entites:

average_precision = np.mean(dataframe_contenu['precision'])
average_recall = np.mean(dataframe_contenu['recall'])
average_f1_score = np.mean(dataframe_contenu['f1_score'])

# Display the averages in a nicely formatted way
print("Average Precision: {:.2f}".format(average_precision))
print("Average Recall: {:.2f}".format(average_recall))
print("Average F1 Score: {:.2f}".format(average_f1_score))

In [ ]:
dataframe_contenu[['precision_relation', 'recall_relation', 'f1_score_relation']] = dataframe_contenu.apply(lambda row: calculate_scores(list(row['entites_relationship'].values()), list(row['entites_relationship_predict'].values())), axis=1, result_type='expand')


In [ ]:
# extract relationship metric
average_precision = np.mean(dataframe_contenu['precision_relation'])
average_recall = np.mean(dataframe_contenu['recall_relation'])
average_f1_score = np.mean(dataframe_contenu['f1_score_relation'])

# Display the averages in a nicely formatted way
print("Average Precision: {:.2f}".format(average_precision))
print("Average Recall: {:.2f}".format(average_recall))
print("Average F1 Score: {:.2f}".format(average_f1_score))

In [1]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

C:\Users\Haboubi\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
